In [1]:
import pandas as pd
from sqlalchemy import null

pd.set_option('display.max_rows', None)

df_backup = pd.read_csv("eta_prediction.csv")
df = df_backup
df.columns = ["Location", "Vehicle", "Shipment Number", "Start-Lat", "Start-Lon", "Dest-Lat", "Dest-Lon", "Time-1", "Time-2", "Time-3", "Time-4", "Time-5"]

for i in range(5):
    column = "Time-"+ str(i+1)
    df[column] = pd.to_datetime(df[column], errors = 'coerce', format='%Y-%m-%d %H:%M:%S')

#df = df[df["Time-4"].notnull()]
#df = df[df["Time-5"].notnull()]

from numpy import datetime64

df["planDurn"] = df["Time-2"].dropna().astype(datetime64)-df["Time-1"].dropna().astype(datetime64)
df["trackDurn"] = df["Time-4"].dropna().astype(datetime64)-df["Time-3"].dropna().astype(datetime64)
df["planRemain"] = df["Time-5"].dropna().astype(datetime64)-df["Time-1"].dropna().astype(datetime64)
df["trackRemain"] = df["Time-5"].dropna().astype(datetime64)-df["Time-3"].dropna().astype(datetime64)
df["Time-25"] = df["Time-5"].dropna().astype(datetime64)-df["Time-2"].dropna().astype(datetime64)

df["Time-45"] = df["Time-5"].dropna().astype(datetime64)-df["Time-4"].dropna().astype(datetime64)


/var/folders/rx/s42f27kn5j5ddqzlhz7qr2yw0000gn/T/ipykernel_6109/443228955.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_backup = pd.read_csv("eta_prediction.csv")


In [2]:
df.shape

(4965694, 18)

In [3]:
df["Location"].unique()

array(['CHN', 'HKG', 'IDN', 'MYS', 'PHL', 'SGP', 'THA', 'TWN'],
      dtype=object)

In [4]:
df[df["Location"]=="HKG"].shape

(330167, 18)

In [5]:
df_CHN = df[df["Location"]=="CHN"]
df_HKG = df[df["Location"]=="HKG"]
df_IDN = df[df["Location"]=="IDN"]
df_MYS = df[df["Location"]=="MYS"]
df_PHL = df[df["Location"]=="PHL"]
df_SGP = df[df["Location"]=="SGP"]
df_THA = df[df["Location"]=="THA"]
df_TWN = df[df["Location"]=="TWN"]
print(
    #df_CHN["Limit"].unique(), 
    df_CHN["Vehicle"].unique().shape,
    #df_HKG["Limit"].unique(), 
    df_HKG["Vehicle"].unique().shape,
    #df_IDN["Limit"].unique(), 
    df_IDN["Vehicle"].unique().shape,
    #df_MYS["Limit"].unique(), 
    df_MYS["Vehicle"].unique().shape,
    #df_PHL["Limit"].unique(), 
    df_PHL["Vehicle"].unique().shape,
    #df_SGP["Limit"].unique(), 
    df_SGP["Vehicle"].unique().shape,
    #df_THA["Limit"].unique(), 
    df_THA["Vehicle"].unique().shape,
    #df_TWN["Limit"].unique(), 
    df_TWN["Vehicle"].unique().shape,
    df["Vehicle"].unique().shape)

#total 211, can assume no duplicate in different region and no reference value 
#use dictionary to deal with the limit type

#model Feb.7, 2017 - Feb.9, 2021 -> Test on the most update data

(4,) (18,) (41,) (18,) (33,) (21,) (20,) (56,) (205,)


In [6]:
df_IDN["Vehicle"].unique()

array(['COLT DIESEL ENGKEL', 'COLT DIESEL DOUBLE', 'WING BOX', 'FUSO',
       '20FT CONTAINER', 'TRONTON', 'CARRY', 'WING BOX DEDICATED', 'L300',
       'LTL TRUCK', 'BIGMAMA', 'WING BOX JUMBO', 'BIGMAMA CURTAIN SLIDER',
       nan, 'CONTAINER 20FT', '40FT CONTAINER', 'CARRY PICK UP',
       'CARRY DEDICATED', 'COLT DIESEL', 'CONTAINER 40FT', 'GRAND MAX',
       'FUSO 8T', 'TRONTON 14T', 'TRONTON 16T', 'FUSO 10T', 'CDD LONG',
       'LCL-TRUCK', 'WING BOX AC', 'FUSO AIR CONDITIONER',
       'COLT DIESEL DOUBLE JUMBO', 'CDE AIR CONDITIONER',
       'WINGBOX AIR CONDITIONER', 'LCL', 'CDD AIR CONDITIONER',
       'COLT DIESEL ENGKEL AIR CONDITIONER', 'LCL LAUT', 'WINGBOX',
       'LCL ONS- 1 DAY SERVICE', 'LCL REGULER - UDARA',
       '20FT CONTAINER AIR CONDITIONER', 'GRAND MAX AIR CONDITIONER'],
      dtype=object)

In [7]:
df[df["Shipment Number"]=="LFL/TMS/TWN.OBW0018616560"]

,Location,Vehicle,Shipment Number,Start-Lat,Start-Lon,Dest-Lat,Dest-Lon,Time-1,Time-2,Time-3,Time-4,Time-5,planDurn,trackDurn,planRemain,trackRemain,Time-25,Time-45
4965689,TWN,NaN,LFL/TMS/TWN.OBW0018616560,24.9,121.113,24.777,121.01,2021-02-09 20:35:00,2021-02-09 21:35:00,2021-02-09 20:35:00,NaT,2021-02-10 23:59:00,0 days 01:00:00,NaT,1 days 03:24:00,1 days 03:24:00,1 days 02:24:00,NaT
4965692,TWN,1T,LFL/TMS/TWN.OBW0018616560,24.9,121.113,24.777,121.01,2021-02-09 21:35:00,2021-02-09 23:35:00,2021-02-09 21:35:00,NaT,2021-02-10 23:59:00,0 days 02:00:00,NaT,1 days 02:24:00,1 days 02:24:00,1 days 00:24:00,NaT


In [8]:
df_HKG_2019 = df_HKG[df_HKG["Time-1"] < "2019-12-31"]["Time-1"]

In [9]:
df_HKG_20191101 = df_HKG[(df_HKG["Time-1"] >= "2019-11-01") & (df_HKG["Time-1"] < "2019-11-02")]
df_HKG_20191101_UniqueTime = df_HKG_20191101["Time-1"].unique()
df_HKG_20191101_UniqueTime.sort()
df_HKG_20191101_UniqueTime

array(['2019-11-01T06:00:00.000000000', '2019-11-01T06:23:00.000000000',
       '2019-11-01T07:00:00.000000000', '2019-11-01T08:00:00.000000000',
       '2019-11-01T09:00:00.000000000', '2019-11-01T09:30:00.000000000',
       '2019-11-01T09:41:59.000000000', '2019-11-01T13:50:00.000000000',
       '2019-11-01T15:24:01.000000000', '2019-11-01T15:57:01.000000000',
       '2019-11-01T16:03:01.000000000', '2019-11-01T16:21:02.000000000',
       '2019-11-01T17:03:01.000000000', '2019-11-01T17:18:01.000000000',
       '2019-11-01T17:24:33.000000000', '2019-11-01T17:45:04.000000000'],
      dtype='datetime64[ns]')

In [10]:
df_HKG_20191101.to_csv("df_HKG_20191101.csv")

In [11]:
df_HKG["Start-Lat"].unique()

array([22.45055, 22.44591, 22.36991, 22.49868, 22.44898, 22.39021,
       22.28007, 22.34338])

In [22]:
df_TWN.to_csv("df_TWN.csv")